### **Código do `MVP` (prototipação)**

> **Contextualização**

O projeto em questão visa facilitar a criação de um gráfico de `Gantt` junto com um currículo em `LaTeX` salvo em `PDF`.

Sendo assim, teremos as respectivas etapas à serem desenvolvidas:

1. `Web Scrapping` das informações do próprio `LinkedIn`
2. Criação do gráfico de `Gantt`
3. Compilação dos arquivos em `LaTeX` para `PDF`.

> **Web Scraping**

* **Bibliotecas**:

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
from dotenv import load_dotenv


* **Configurações:**

In [2]:
# Pasta dos códigos
path_codigos = os.path.dirname(os.getcwd())

# Pasta das entradas
path_entradas = os.path.join(path_codigos, 'Entrada')

# Apontamento para a pasta de entrada
os.chdir(path_entradas)

# Carregando as variáveis de ambiente
load_dotenv('credenciais.env')

# # Selenium de forma oculta
# configuracoes = webdriver.ChromeOptions()
# configuracoes.add_argument("--headless=new")


True

* **Links:**

In [3]:
# Usuário desejado
nome_perfil = os.environ['perfil']

# URL padrão
url_linkedin = 'https://www.linkedin.com/'

# Link direto para o perfil
url_perfil = url_linkedin + 'in' + f'/{nome_perfil}/'

# Página de Login
url_login = url_linkedin + '/login/pt'

# Detalhes
url_detalhes = url_perfil + 'details/'

# Seções
url_experiencias = url_detalhes + 'experience'
url_estudos = url_detalhes + 'education'
url_certificados = url_detalhes + 'certifications'
url_voluntarios = url_detalhes + 'volunteering-experiences'

* **Navegador:**

In [4]:
# Criando o navegador
navegador = webdriver.Chrome()  # options=configuracoes

# Adicionando delay de carregamento
time.sleep(1)


* **Login:**

In [17]:
# Acessando o local de login
navegador.get(url_login)

# TODO: Remover sleeps e, adicionar algum wait da página carregada

# Adicionando delay de carregamento
time.sleep(1)

# Preenchendo o email
navegador.find_element(By.ID, 'username').send_keys(
    os.environ['usuario']
)

# Preenchendo a senha
navegador.find_element(By.ID, 'password').send_keys(
    os.environ['senha']
)

# Entrando
navegador.find_element(By.ID, 'password').send_keys(Keys.RETURN)

# Adicionando delay devido aprovação de login
time.sleep(10)


* **Perfil**:

In [27]:
def get_nome(html):
    '''
    Coleta o nome do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Nome do perfil
    nome = html.find('h1').text

    # Removendo espaçamentos exagerados
    nome = nome.strip()

    # Retornando o nome do profissional
    return nome


In [28]:
def get_frase(html):
    '''
    Coleta a frase de efeito do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Frase de efeito
    frase = html.find('div', {'class': 'text-body-medium'}).text

    # Removendo espaçamentos exagerados
    frase = frase.strip()

    # Retornando a frase de efeito
    return frase


In [39]:
def get_sobre(html):
    '''
    Coleta o sobre do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Filtro dos spans
    spans = html.find_all('span')

    # Variável auxiliar para contador de, onde está o sobre
    index_aux = 0

    # Iterando a lista dos spans até encontrar qual o index está o Sobre
    for index, div in enumerate(spans):

        # Se encontrar o sobre
        if 'Sobre' in div:

            # Atribui o índice
            indice = index

    # Pegando o Sobre
    sobre = spans[indice + 2].text

    # Removendo espaçamentos exagerados
    sobre = sobre.strip()

    # Retornando o sobre
    return sobre

In [51]:
# Acessando o perfil do usuários
navegador.get(url=url_perfil)

# Adicionando delay de carregamento
time.sleep(3)

# Pegando o conteúdo HTML do perfil
html_perfil = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

# Nome do perfil
label_nome = get_nome(html_perfil)

# Frase de efeito
label_frase = get_frase(html_perfil)

# Sobre
label_sobre = get_sobre(html_perfil)

# Informativo
print(
    '\n> [label_nome]:', '\n\t', label_nome, '\n'
    '\n> [label_frase]:', '\n\t', label_frase, '\n'
    '\n> [label_sobre]:', '\n\t', label_sobre, '\n'
)



> [label_nome]: 
	 Murilo Chaves Jayme 

> [label_frase]: 
	 Cientista de dados com foco em mercado financeiro 

> [label_sobre]: 
	 Bacharel em Sistemas de Informação pela ULBRA e atualmente cursando dois programas de pós-graduação: MBA em Big Data Science pela FIAP e MBA em Ciência de Dados para o Mercado Financeiro na XP Educação. Tem experiência como pesquisador em projetos financiados pela FAPEG. E, atualmente buscando oportunidades (Cientista de dados ou Analista Quantitativo) para atuação com pesquisa e inovação voltadas para o mercado financeiro. 



* **Experiência:**

In [52]:
# Acessando a página de experiências profissionais
navegador.get(url=url_experiencias)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
experiencias = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

* **Formação acadêmica:**

In [53]:
# Acessando a página dos estudos
navegador.get(url=url_estudos)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
estudos = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

* **Licenças e Certificados:**

In [54]:
# Acessando a página dos certificados
navegador.get(url=url_certificados)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
certificados = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

* **Trabalho voluntário:**

In [55]:
# Acessando a página dos voluntariados
navegador.get(url=url_voluntarios)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
voluntarios = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)